cit.014	Human Settlements

Description: http://ghsl.jrc.ec.europa.eu/ghs_bu.php 
Download as http://ghsl.jrc.ec.europa.eu/datasets.php

GHS_BUILT_LDS1975_GLOBE_R2016A_54009_250 (215MB)
GHS_BUILT_LDS1990_GLOBE_R2016A_3857_38 (854MB) 
GHS_BUILT_LDS2000_GLOBE_R2016A_3857_38 (892MB) 
GHS_BUILT_LDS2014_GLOBE_R2016A_3857_38 (900MB)

File type: Geotiff

In [1]:
# Libraries for downloading data from remote server (may be ftp)
import requests
from urllib.request import urlopen
from contextlib import closing
import shutil

# Library for uploading/downloading data to/from S3
import boto3

# Libraries for handling data
import rasterio as rio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio import crs

import numpy as np
# from netCDF4 import Dataset
# import pandas as pd
# import scipy

# Libraries for various helper functions
# from datetime import datetime
import os
import threading
import sys
from glob import glob

Define local file locations

In [2]:
local_folder = "/Users/Max81007/Desktop/Python/Resource_Watch/Raster/cit_014/"

sub_folder1 = "GHS_BUILT_LDS1975_GLOBE_R2016A_54009_250_v1_0/"
sub_folder2 = "GHS_BUILT_LDS2000_GLOBE_R2016A_3857_38_v1_0/"
sub_folder3 = "GHS_BUILT_LDS2014_GLOBE_R2016A_3857_38_v1_0/"
sub_folder4 = "GHS_BUILT_LDS1990_GLOBE_R2016A_3857_38_v1_0/"

file_name1 = "GHS_BUILT_LDS1975_GLOBE_R2016A_54009_250_v1_0"
file_name2 = "GHS_BUILT_LDS2000_GLOBE_R2016A_3857_38_v1_0_p1"
file_name3 = "GHS_BUILT_LDS2000_GLOBE_R2016A_3857_38_v1_0_p2"
file_name4 = "GHS_BUILT_LDS2014_GLOBE_R2016A_3857_38_v1_0_p1"
file_name5 = "GHS_BUILT_LDS2014_GLOBE_R2016A_3857_38_v1_0_p2"
file_name6 = "GHS_BUILT_LDS1990_GLOBE_R2016A_3857_38_v1_0_p1"
file_name7 = "GHS_BUILT_LDS1990_GLOBE_R2016A_3857_38_v1_0_p2"


local_orig1 = local_folder+ sub_folder1 + file_name1 + ".tif" 
local_orig2 = local_folder+ sub_folder2 + file_name2 + ".tif"  
local_orig3 = local_folder+ sub_folder2 + file_name3 + ".tif"  
local_orig4 = local_folder+ sub_folder3 + file_name4 + ".tif"  
local_orig5 = local_folder+ sub_folder3 + file_name5 + ".tif"  
local_orig6 = local_folder+ sub_folder4 + file_name6 + ".tif"  
local_orig7 = local_folder+ sub_folder4 + file_name7 + ".tif" 
# local_orig8 = local_folder+ sub_folder4 + file_name8 + ".tif" 

orig_extension_length = 4 #4 for each char in .tif

local_edit1 = local_orig1[:-orig_extension_length] + "edit.tif" 
local_edit2 = local_orig2[:-orig_extension_length] + "edit.tif" 
local_edit3 = local_orig3[:-orig_extension_length] + "edit.tif" 
local_edit4 = local_orig4[:-orig_extension_length] + "edit.tif" 
local_edit5 = local_orig5[:-orig_extension_length] + "edit.tif" 
local_edit6 = local_orig6[:-orig_extension_length] + "edit.tif" 
local_edit7 = local_orig7[:-orig_extension_length] + "edit.tif" 

# upload_name1 = "maize2_tot_days"
# upload_name2 = "maize2_plant_start"
# upload_name3 = "maize_tot_days"
# upload_name4 = "maize_plant_start"
# upload_name5 = "rice2_tot_days"
# upload_name6 = "rice2_plant_start"
# upload_name7 = "rice_tot_days"
# upload_name8 = "rice_plant_start"
# upload_name9 = "soybeans_tot_days"
# upload_name10 = "soybeans_plant_start"
# upload_name11 = "wheat_tot_days"
# upload_name12 = "wheat_plant_start"
# upload_name13 = "wheat_winter_tot_days"
# upload_name14 = "wheat_winter_plant_start"

upload_name1 = "cit_014_1975_builtup_grid"
upload_name2 = "cit_014_2000_builtup_grid_p1"
upload_name3 = "cit_014_2000_builtup_grid_p2"
upload_name4 = "cit_014_2014_builtup_grid_p1"
upload_name5 = "cit_014_2014_builtup_grid_p2"
upload_name6 = "cit_014_1990_builtup_grid_p1"
upload_name7 = "cit_014_1990_builtup_grid_p2"


s3

In [3]:
s3_upload = boto3.client("s3")
s3_download = boto3.resource("s3")

s3_bucket = "wri-public-data"
s3_folder = "resourcewatch/raster/cit_014_human_settlements/"

s3_key_orig1 = s3_folder + upload_name1 + ".tif" 
s3_key_edit1 = s3_key_orig1[0:-4] + "_edit.tif"

s3_key_orig2 = s3_folder + upload_name2 + ".tif" 
s3_key_edit2 = s3_key_orig2[0:-4] + "_edit.tif"

s3_key_orig3 = s3_folder + upload_name3 + ".tif" 
s3_key_edit3 = s3_key_orig3[0:-4] + "_edit.tif"

s3_key_orig4 = s3_folder + upload_name4 + ".tif" 
s3_key_edit4 = s3_key_orig4[0:-4] + "_edit.tif"

s3_key_orig5 = s3_folder + upload_name5 + ".tif" 
s3_key_edit5 = s3_key_orig5[0:-4] + "_edit.tif"

s3_key_orig6= s3_folder + upload_name6 + ".tif" 
s3_key_edit6 = s3_key_orig6[0:-4] + "_edit.tif"

s3_key_orig7 = s3_folder + upload_name7 + ".tif" 
s3_key_edit7 = s3_key_orig7[0:-4] + "_edit.tif"

# s3_key_orig8 = s3_folder + upload_name8 + ".tif" 
# s3_key_edit8 = s3_key_orig8[0:-4] + "_edit.tif"

# s3_key_orig9 = s3_folder + upload_name9 + ".tif" 
# s3_key_edit9 = s3_key_orig9[0:-4] + "_edit.tif"

# s3_key_orig10 = s3_folder + upload_name10 + ".tif" 
# s3_key_edit10 = s3_key_orig10[0:-4] + "_edit.tif"

# s3_key_orig11 = s3_folder + upload_name11 + ".tif" 
# s3_key_edit11 = s3_key_orig11[0:-4] + "_edit.tif"

# s3_key_orig12 = s3_folder + upload_name12 + ".tif" 
# s3_key_edit12 = s3_key_orig12[0:-4] + "_edit.tif"

# s3_key_orig13 = s3_folder + upload_name13 + ".tif" 
# s3_key_edit13 = s3_key_orig13[0:-4] + "_edit.tif"

# s3_key_orig14= s3_folder + upload_name14 + ".tif" 
# s3_key_edit14 = s3_key_orig14[0:-4] + "_edit.tif"

class ProgressPercentage(object):
        def __init__(self, filename):
            self._filename = filename
            self._size = float(os.path.getsize(filename))
            self._seen_so_far = 0
            self._lock = threading.Lock()

        def __call__(self, bytes_amount):
            # To simplify we'll assume this is hooked up
            # to a single filename.
            with self._lock:
                self._seen_so_far += bytes_amount
                percentage = (self._seen_so_far / self._size) * 100
                sys.stdout.write("\r%s  %s / %s  (%.2f%%)"%(
                        self._filename, self._seen_so_far, self._size,
                        percentage))
                sys.stdout.flush()

In [4]:
band_ids = [upload_name1, upload_name2, upload_name3, upload_name4, upload_name5, upload_name6, upload_name7]
merge_name = "cit_014_human_settlements.tif"
s3_key_merge = s3_folder + merge_name

In [5]:
os.getcwd()
os.chdir(local_folder)
os.environ["local_orig1"] = local_orig1
os.environ["local_edit1"] = local_edit1
os.environ["local_orig2"] = local_orig2
os.environ["local_edit2"] = local_edit2
os.environ["local_orig3"] = local_orig3
os.environ["local_edit3"] = local_edit3
os.environ["local_orig4"] = local_orig4
os.environ["local_edit4"] = local_edit4
os.environ["local_orig5"] = local_orig5
os.environ["local_edit5"] = local_edit5
os.environ["local_orig6"] = local_orig6
os.environ["local_edit6"] = local_edit6
os.environ["local_orig7"] = local_orig7
os.environ["local_edit7"] = local_edit7

In [6]:
!echo %local_orig1%
!echo %local_edit1%

/Users/Max81007/Desktop/Python/Resource_Watch/Raster/cit_014/GHS_BUILT_LDS1975_GLOBE_R2016A_54009_250_v1_0/GHS_BUILT_LDS1975_GLOBE_R2016A_54009_250_v1_0.tif
/Users/Max81007/Desktop/Python/Resource_Watch/Raster/cit_014/GHS_BUILT_LDS1975_GLOBE_R2016A_54009_250_v1_0/GHS_BUILT_LDS1975_GLOBE_R2016A_54009_250_v1_0edit.tif


In [12]:
print1 ="gdalwarp -overwrite -t_srs epsg:4326 -srcnodata none -co compress=lzw -co TILED=YES -co BIGTIFF=YES" + " " +local_orig1 +" "+ local_edit1
print(print1)

gdalwarp -overwrite -t_srs epsg:4326 -srcnodata none -co compress=lzw -co TILED=YES -co BIGTIFF=YES /Users/Max81007/Desktop/Python/Resource_Watch/Raster/cit_014/GHS_BUILT_LDS1975_GLOBE_R2016A_54009_250_v1_0/GHS_BUILT_LDS1975_GLOBE_R2016A_54009_250_v1_0.tif /Users/Max81007/Desktop/Python/Resource_Watch/Raster/cit_014/GHS_BUILT_LDS1975_GLOBE_R2016A_54009_250_v1_0/GHS_BUILT_LDS1975_GLOBE_R2016A_54009_250_v1_0edit.tif


In [10]:
local_orig = [local_orig1, local_orig2, local_orig3, local_orig4, local_orig5, local_orig6, local_orig7]
local_edit = [local_edit1, local_edit2, local_edit3, local_edit4, local_edit5, local_edit6, local_edit7]
printlist = "gdalwarp -overwrite -t_srs epsg:4326 -srcnodata none -co compress=lzw" + " " + str(local_orig)+ " " +str(local_edit)
print(printlist)

gdalwarp -overwrite -t_srs epsg:4326 -srcnodata none -co compress=lzw ['/Users/Max81007/Desktop/Python/Resource_Watch/Raster/cit_014/GHS_BUILT_LDS1975_GLOBE_R2016A_54009_250_v1_0/GHS_BUILT_LDS1975_GLOBE_R2016A_54009_250_v1_0.tif', '/Users/Max81007/Desktop/Python/Resource_Watch/Raster/cit_014/GHS_BUILT_LDS2000_GLOBE_R2016A_3857_38_v1_0/GHS_BUILT_LDS2000_GLOBE_R2016A_3857_38_v1_0_p1.tif', '/Users/Max81007/Desktop/Python/Resource_Watch/Raster/cit_014/GHS_BUILT_LDS2000_GLOBE_R2016A_3857_38_v1_0/GHS_BUILT_LDS2000_GLOBE_R2016A_3857_38_v1_0_p2.tif', '/Users/Max81007/Desktop/Python/Resource_Watch/Raster/cit_014/GHS_BUILT_LDS2014_GLOBE_R2016A_3857_38_v1_0/GHS_BUILT_LDS2014_GLOBE_R2016A_3857_38_v1_0_p1.tif', '/Users/Max81007/Desktop/Python/Resource_Watch/Raster/cit_014/GHS_BUILT_LDS2014_GLOBE_R2016A_3857_38_v1_0/GHS_BUILT_LDS2014_GLOBE_R2016A_3857_38_v1_0_p2.tif', '/Users/Max81007/Desktop/Python/Resource_Watch/Raster/cit_014/GHS_BUILT_LDS1990_GLOBE_R2016A_3857_38_v1_0/GHS_BUILT_LDS1990_GLOBE_R20

In [15]:
print1 ="rio warp" + " " +local_orig1 +" "+ local_edit1 + " "+"--force-overwrite --dst-crs epsg:4326 --co compress=LZW --co tiled=true --co blockxsize=256 --co blockysize=256"
print(print1)

rio warp /Users/Max81007/Desktop/Python/Resource_Watch/Raster/cit_014/GHS_BUILT_LDS1975_GLOBE_R2016A_54009_250_v1_0/GHS_BUILT_LDS1975_GLOBE_R2016A_54009_250_v1_0.tif /Users/Max81007/Desktop/Python/Resource_Watch/Raster/cit_014/GHS_BUILT_LDS1975_GLOBE_R2016A_54009_250_v1_0/GHS_BUILT_LDS1975_GLOBE_R2016A_54009_250_v1_0edit.tif --force-overwrite --dst-crs epsg:4326 --co compress=LZW --co tiled=true --co blockxsize=256 --co blockysize=256


In [ ]:
!gdalwarp -overwrite -t_srs epsg:4326 -srcnodata none -co compress=lzw %local_orig2% %local_edit2%


In [ ]:
!gdalwarp -overwrite -t_srs epsg:4326 -srcnodata none -co compress=lzw %local_orig3% %local_edit3%


In [ ]:
!gdalwarp -overwrite -t_srs epsg:4326 -srcnodata none -co compress=lzw %local_orig4% %local_edit4%


In [ ]:
!gdalwarp -overwrite -t_srs epsg:4326 -srcnodata none -co compress=lzw %local_orig5% %local_edit5%


In [ ]:
!gdalwarp -overwrite -t_srs epsg:4326 -srcnodata none -co compress=lzw %local_orig6% %local_edit6%


In [ ]:
!gdalwarp -overwrite -t_srs epsg:4326 -srcnodata none -co compress=lzw %local_orig7% %local_edit7%

In [ ]:
merge_files = [local_edit1, local_edit2, local_edit3, local_edit4, local_edit5, local_edit6, local_edit7]
tmp_merge = local_folder + merge_name


Use rasterio to reproject and compress

In [11]:
files = [local_orig1, local_orig2, local_orig3, local_orig4, local_orig5, local_orig6, local_orig7]
for file in files:
    with rio.open(file, 'r') as src:
        profile = src.profile
        print(profile)

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.402823e+38, 'width': 141988, 'height': 60942, 'count': 1, 'crs': CRS({'proj': 'moll', 'lon_0': 0, 'x_0': 0, 'y_0': 0, 'ellps': 'WGS84', 'units': 'm', 'no_defs': True}), 'transform': Affine(250.0, 0.0, -17619594.54744353,
       0.0, -250.0, 8750529.46186849), 'blockxsize': 512, 'blockysize': 512, 'tiled': True, 'compress': 'lzw', 'interleave': 'band'}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': 0.0, 'width': 507904, 'height': 647168, 'count': 1, 'crs': CRS({'proj': 'merc', 'a': 6378137, 'b': 6378137, 'lat_ts': 0, 'lon_0': 0, 'x_0': 0, 'y_0': 0, 'k': 1, 'units': 'm', 'nadgrids': '@null', 'wktext': True, 'no_defs': True}), 'transform': Affine(38.21851414259241, 0.0, -20037508.342789244,
       0.0, -38.21851414259243, 16906647.664228424), 'blockxsize': 4096, 'blockysize': 4096, 'tiled': True, 'compress': 'lzw', 'interleave': 'band'}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': 0.0, 'width': 581632, 'height': 593920, 'count': 1, 

In [ ]:
with rio.open(merge_files[0]) as src:
    kwargs = src.profile
    
kwargs.update(
    count=len(merge_files)
)
    
with rio.open(tmp_merge, 'w', **kwargs) as dst:
    for idx, file in enumerate(merge_files):
        print(idx)
        with rio.open(file) as src:

            band = idx+1
            windows = src.block_windows()

            for win_id, window in windows:
                src_data = src.read(1, window=window)
                dst.write_band(band, src_data, window=window)


Upload orig and edit files to s3

In [ ]:
# Original
s3_upload.upload_file(local_orig1, s3_bucket, s3_key_orig1,
                         Callback=ProgressPercentage(local_orig1))
s3_upload.upload_file(local_orig3, s3_bucket, s3_key_orig3,
                         Callback=ProgressPercentage(local_orig3))
s3_upload.upload_file(local_orig5, s3_bucket, s3_key_orig5,
                         Callback=ProgressPercentage(local_orig5))
s3_upload.upload_file(local_orig7, s3_bucket, s3_key_orig7,
                         Callback=ProgressPercentage(local_orig7))
s3_upload.upload_file(local_orig9, s3_bucket, s3_key_orig9,
                         Callback=ProgressPercentage(local_orig9))
s3_upload.upload_file(local_orig11, s3_bucket, s3_key_orig11,
                         Callback=ProgressPercentage(local_orig11))
s3_upload.upload_file(local_orig13, s3_bucket, s3_key_orig13,
                         Callback=ProgressPercentage(local_orig13))


# Edit
s3_upload.upload_file(local_edit1, s3_bucket, s3_key_edit1,
                         Callback=ProgressPercentage(local_edit1))
s3_upload.upload_file(local_edit3, s3_bucket, s3_key_edit3,
                         Callback=ProgressPercentage(local_edit3))
s3_upload.upload_file(local_edit5, s3_bucket, s3_key_edit5,
                         Callback=ProgressPercentage(local_edit5))
s3_upload.upload_file(local_edit7, s3_bucket, s3_key_edit7,
                         Callback=ProgressPercentage(local_edit7))
s3_upload.upload_file(local_edit9, s3_bucket, s3_key_edit9,
                         Callback=ProgressPercentage(local_edit9))
s3_upload.upload_file(local_edit11, s3_bucket, s3_key_edit11,
                         Callback=ProgressPercentage(local_edit11))
s3_upload.upload_file(local_edit13, s3_bucket, s3_key_edit13,
                         Callback=ProgressPercentage(local_edit13))

# Merged 

s3_upload.upload_file(tmp_merge, s3_bucket, s3_key_merge,
           Callback=ProgressPercentage(tmp_merge))

In [ ]:
os.environ["Zs3_key"] = "s3://wri-public-data/" + s3_key_merge
os.environ["Zgs_key"] = "gs://resource-watch-public/" + s3_key_merge


In [ ]:
!gsutil cp %Zs3_key% %Zgs_key%


In [ ]:
os.environ["asset_id"] = "users/resourcewatch/cit_014_human_settlements"
!earthengine upload image --asset_id=%asset_id% %Zgs_key%


In [ ]:
os.environ["band_names"] = str(band_ids)
!earthengine asset set -p band_names="%band_names%" %asset_id%
